In [ ]:
%%capture
!pip install pecanpy
!pip install numpy==1.22

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

import scipy.stats as st

import os
import subprocess

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/graph_analytics/ThaiCapitalMarket/GraphEmbeding/Data/ThaiCapitalMktLink-TimeSeries.csv")
dfx = df.drop(columns = ['Date']).copy()

# [1] Pearson Correlarion

In [ ]:
corr_list = ['pearson', 'spearman', 'kendall']

In [ ]:
for i, corr in enumerate(corr_list):
    dfx_corr = dfx.corr(method = corr)
    dfx_corr = pd.melt(dfx_corr, ignore_index = False).reset_index()
    dfx_corr.columns = ['var1', 'var2', corr]

    if i == 0:
        dfx2 = dfx_corr.copy()
    else:
        dfx2 = dfx2.merge(dfx_corr, on = ['var1', 'var2'])

In [ ]:
# map_var = {e:i+1 for i, e in enumerate(dfx2['var1'].unique())}
# inv_map_var = {i+1:e for i, e in enumerate(dfx2['var1'].unique())}

In [ ]:
map_var = {'SET': 1,
        'MAI': 2,
        'ZeroShort': 3,
        'ZeroLong': 4,
        'CorpBond': 5,
        'THB': 6,
        'EquityFlow': 7,
        'BondFlow': 8,
        'EMAsiaEquity': 9,
        'SP500': 10,
        'EMBond': 11,
        'USBond': 12,
        'EMAsiaFX': 13,
        'USD': 14}

inv_map_var = {i: e for e, i in map_var.items()}

In [ ]:
dfx2['var1'] = dfx2['var1'].map(map_var)
dfx2['var2'] = dfx2['var2'].map(map_var)

In [ ]:
dfx3 = dfx2[dfx2['var1'] < dfx2['var2']].copy()

dfx3['pearson'] = dfx3['pearson'].round(6)
dfx3['spearman'] = dfx3['spearman'].round(6)
dfx3['kendall'] = dfx3['kendall'].round(6)

In [ ]:
dfx3 = dfx3.sort_values(['var1', 'var2'], ignore_index=True)

In [ ]:
path_save = "Data/input"
os.makedirs(path_save, exist_ok=True)

In [ ]:
for corr in corr_list:
    dfx3[['var1', 'var2', corr]].to_csv(f"{path_save}/{corr.capitalize()}_Weighted.edg", sep = '\t', header = False, index = False)
    dfx3[dfx3[corr] > 0][['var1', 'var2']].to_csv(f"{path_save}/{corr.capitalize()}_Positive.edg", sep = '\t', header = False, index = False)
    dfx3[dfx3[corr].abs() > 0.1][['var1', 'var2']].to_csv(f"{path_save}/{corr.capitalize()}_AbsGreater005.edg", sep = '\t', header = False, index = False)
    dfx3[dfx3[corr].abs() > 0.1][['var1', 'var2']].to_csv(f"{path_save}/{corr.capitalize()}_AbsGreater01.edg", sep = '\t', header = False, index = False)
    dfx3[dfx3[corr].abs() > 0.2][['var1', 'var2']].to_csv(f"{path_save}/{corr.capitalize()}_AbsGreater02.edg", sep = '\t', header = False, index = False)
    dfx3[dfx3[corr].abs() > 0.5][['var1', 'var2']].to_csv(f"{path_save}/{corr.capitalize()}_AbsGreater05.edg", sep = '\t', header = False, index = False)

In [ ]:
path_output = "Data/output"
os.makedirs(path_output, exist_ok=True)

In [ ]:
mode = "DenseOTF"
input_path = "/content/Data/input"
output_path = "/content/Data/output"

for dim in [2, 3]:
    for corr in corr_list:
        for e in ['Weighted', 'Positive', 'AbsGreater005', 'AbsGreater01', 'AbsGreater02', 'AbsGreater05']:
            if e == 'Weighted':
                command_str = f"pecanpy --input {input_path}/{corr.capitalize()}_{e}.edg --output {output_path}/{corr.capitalize()}_{e}_dim{dim}.emb --mode {mode} --dimensions {dim} --weighted --extend"
            else:
                command_str = f"pecanpy --input {input_path}/{corr.capitalize()}_{e}.edg --output {output_path}/{corr.capitalize()}_{e}_dim{dim}.emb --mode {mode} --dimensions {dim}"
            
            output = subprocess.run(command_str, shell=True, stdout=subprocess.PIPE, text=True)
            print(output.stdout)

Took 00:00:00.00 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:19.23 to generate walks
Took 00:00:00.04 to train embeddings

Took 00:00:00.00 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:10.12 to generate walks
Took 00:00:00.02 to train embeddings

Took 00:00:00.00 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:08.06 to generate walks
Took 00:00:00.02 to train embeddings

Took 00:00:00.00 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:07.03 to generate walks
Took 00:00:00.06 to train embeddings

Took 00:00:00.00 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:06.27 to generate walks
Took 00:00:00.02 to train embeddings

Took 00:00:00.00 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:07.95 to generate walks
Took 00:00:00.02 to train embeddings

Took 00:00:00.00 to load Graph
Too

In [ ]:
list_emb = os.listdir(output_path)

In [ ]:
dfx_corr2 = dfx_corr.copy()

In [ ]:
dfx_corr2

,var1,var2,kendall
0,SET,SET,1.000000
1,MAI,SET,0.459500
2,ZeroShort,SET,0.014317
3,ZeroLong,SET,0.019098
4,CorpBond,SET,-0.008721
...,...,...,...
191,SP500,USD,-0.211699
192,EMBond,USD,-0.142966
193,USBond,USD,-0.058428
194,EMAsiaFX,USD,-0.486064


In [ ]:
k = dfx.corr(method = corr)

In [ ]:
k

,SET,MAI,ZeroShort,ZeroLong,CorpBond,THB,EquityFlow,BondFlow,EMAsiaEquity,SP500,EMBond,USBond,EMAsiaFX,USD
SET,1.000000,0.459500,0.014317,0.019098,-0.008721,0.150046,0.207731,0.035848,0.360988,0.131344,0.171453,-0.020311,0.140979,-0.062647
MAI,0.459500,1.000000,-0.005811,0.000341,-0.001637,0.098107,0.097303,0.054235,0.194358,0.080492,0.108918,0.002277,0.074893,-0.030199
ZeroShort,0.014317,-0.005811,1.000000,0.252695,-0.101729,0.000821,0.022639,-0.007854,-0.004194,0.005798,-0.012645,-0.023942,0.014810,-0.007973
ZeroLong,0.019098,0.000341,0.252695,1.000000,-0.454801,-0.064769,-0.014525,-0.072184,0.020664,0.024776,-0.078408,-0.084581,-0.016297,0.020889
CorpBond,-0.008721,-0.001637,-0.101729,-0.454801,1.000000,0.040786,-0.017731,0.042276,-0.014700,-0.033249,0.017977,0.046856,0.005572,-0.007751
THB,0.150046,0.098107,0.000821,-0.064769,0.040786,1.000000,0.079868,0.049438,0.187416,0.151173,0.157498,0.044101,0.448018,-0.326792
EquityFlow,0.207731,0.097303,0.022639,-0.014525,-0.017731,0.079868,1.000000,0.100548,0.175150,0.019187,0.140646,0.031331,0.038326,-0.012763
BondFlow,0.035848,0.054235,-0.007854,-0.072184,0.042276,0.049438,0.100548,1.000000,0.049405,0.008210,0.074937,0.014410,0.020371,-0.013343
EMAsiaEquity,0.360988,0.194358,-0.004194,0.020664,-0.014700,0.187416,0.175150,0.049405,1.000000,0.179048,0.255321,-0.025686,0.254896,-0.102439
SP500,0.131344,0.080492,0.005798,0.024776,-0.033249,0.151173,0.019187,0.008210,0.179048,1.000000,0.098814,-0.188609,0.293820,-0.211699


In [ ]:
dict_df = {}
for corr in corr_list:
    dfx_corr = dfx.corr(method = corr)
    dfx_corr.values[[np.arange(dfx_corr.shape[0])]*2] = 0
    for e in ['Weighted', 'Positive', 'AbsGreater005', 'AbsGreater01', 'AbsGreater02', 'AbsGreater05']:

        if e == 'Weighted':
            dfx_corr2 = dfx_corr.copy()
        elif e == 'Positive':
            dfx_corr2 = (dfx_corr > 0).astype(int).copy()
        elif e == 'AbsGreater005':
            dfx_corr2 = (dfx_corr.abs() > 0.05).astype(int).copy()
        elif e == 'AbsGreater01':
            dfx_corr2 = (dfx_corr.abs() > 0.1).astype(int).copy()
        elif e == 'AbsGreater02':
            dfx_corr2 = (dfx_corr.abs() > 0.2).astype(int).copy()
        elif e == 'AbsGreater05':
            dfx_corr2 = (dfx_corr.abs() > 0.5).astype(int).copy()

        for dim in [2, 3]:
            filename = f"{corr.capitalize()}_{e}_dim{dim}.emb"
            if filename in list_emb:
                df_temp = pd.read_csv(f"{output_path}/{filename}", delimiter=' ', skiprows = 1, header = None, index_col = [0])
                df_temp.columns = [f'N2VP_dim{i}of{dim}' for i in range(1, dim+1)]

                # df_temp = df_temp.reset_index().rename(columns = {'index':'var', 0:'var'})
                # df_temp['var'] = df_temp['var'].map(inv_map_var)
                df_temp.index = df_temp.index.map(inv_map_var)

                dfx_corr2 = dfx_corr2.merge(df_temp, left_index=True, right_index=True, how='left')
            else:
                df_temp = pd.DataFrame(list(map_var.keys())).set_index(0)
                for i in range(1, dim+1):
                    col = f'N2VP_dim{i}of{dim}'
                    df_temp[col] = np.nan
                dfx_corr2 = dfx_corr2.merge(df_temp, left_index=True, right_index=True, how='left')

        dfx_corr2.index.name = f"{corr.capitalize()}_{e}"
        dict_df[f"{corr.capitalize()}_{e}"] = dfx_corr2

<ipython-input-105-01098edc755a>:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dfx_corr.values[[np.arange(dfx_corr.shape[0])]*2] = 0


In [ ]:
# with pd.ExcelWriter(f"Data/Embedding.xlsx") as writer:
#     for sheet_name, df in dict_df.items():
#         df.to_excel(writer, sheet_name=sheet_name)

In [ ]:
dict_df.keys()

dict_keys(['Pearson_Weighted', 'Pearson_Positive', 'Pearson_AbsGreater005', 'Pearson_AbsGreater01', 'Pearson_AbsGreater02', 'Pearson_AbsGreater05', 'Spearman_Weighted', 'Spearman_Positive', 'Spearman_AbsGreater005', 'Spearman_AbsGreater01', 'Spearman_AbsGreater02', 'Spearman_AbsGreater05', 'Kendall_Weighted', 'Kendall_Positive', 'Kendall_AbsGreater005', 'Kendall_AbsGreater01', 'Kendall_AbsGreater02', 'Kendall_AbsGreater05'])

# [2] Granger

In [ ]:
df_granger = pd.read_excel("/content/drive/MyDrive/graph_analytics/ThaiCapitalMarket/GraphEmbeding/Data/Granger_Sig.xlsx")

In [ ]:
df_granger

,Y,X,Granger_6D,Granger_11D,Granger_VL
0,Thai-Equity-Set,Thai-Equity-MAI,False,False,False
1,Thai-Equity-MAI,Thai-Equity-Set,False,False,False
2,Thai-Equity-Set,Thai-Zero-Short,False,False,False
3,Thai-Zero-Short,Thai-Equity-Set,False,False,False
4,Thai-Equity-Set,Thai-Zero-Long,False,False,False
...,...,...,...,...,...
177,EM-Asia-FX,US-Bond,False,False,False
178,US-Bond,US-FX,False,False,True
179,US-FX,US-Bond,False,False,True
180,EM-Asia-FX,US-FX,False,False,True


In [ ]:
map_var2 = {'Thai-Equity-Set': 1,
            'Thai-Equity-MAI': 2,
            'Thai-Zero-Short': 3,
            'Thai-Zero-Long': 4,
            'Thai-Bond-Corp': 5,
            'Thai-FX': 6,
            'Thai-Equity-Flow': 7,
            'Thai-Bond-Flow-NR': 8,
            'EM-Asia-Equity': 9,
            'US-Equity': 10,
            'EM-Bond': 11,
            'US-Bond': 12,
            'EM-Asia-FX': 13,
            'US-FX': 14}

In [ ]:
# df_granger['var1'] = df_granger['Y'].map(map_var2)
# df_granger['var2'] = df_granger['X'].map(map_var2)

df_granger['var1'] = df_granger['X'].map(map_var2)
df_granger['var2'] = df_granger['Y'].map(map_var2)

In [ ]:
df_granger = df_granger.sort_values(['var1', 'var2'], ignore_index=True)

In [ ]:
df_granger

,Y,X,Granger_6D,Granger_11D,Granger_VL,var1,var2
0,Thai-Equity-MAI,Thai-Equity-Set,False,False,False,1,2
1,Thai-Zero-Short,Thai-Equity-Set,False,False,False,1,3
2,Thai-Zero-Long,Thai-Equity-Set,False,False,False,1,4
3,Thai-Bond-Corp,Thai-Equity-Set,False,False,False,1,5
4,Thai-FX,Thai-Equity-Set,False,False,False,1,6
...,...,...,...,...,...,...,...
177,EM-Asia-Equity,US-FX,True,True,True,14,9
178,US-Equity,US-FX,False,False,True,14,10
179,EM-Bond,US-FX,True,True,True,14,11
180,US-Bond,US-FX,False,False,True,14,12


In [ ]:
path_save = "Data/input"
os.makedirs(path_save, exist_ok=True)

In [ ]:
granger_list = ['Granger_6D', 'Granger_11D', 'Granger_VL']

In [ ]:
for col in granger_list:
    df_granger[df_granger[col]][['var1', 'var2']].to_csv(f"{path_save}/{col}.edg", sep = '\t', header = False, index = False)

In [ ]:
path_output = "Data/output"
os.makedirs(path_output, exist_ok=True)

In [ ]:
mode = "DenseOTF"
input_path = "/content/Data/input"
output_path = "/content/Data/output"

for dim in [2, 3]:
    for col in granger_list:
        command_str = f"pecanpy --input {input_path}/{col}.edg --output {output_path}/{col}_dim{dim}.emb --mode {mode} --dimensions {dim} --directed"
        
        output = subprocess.run(command_str, shell=True, stdout=subprocess.PIPE, text=True)
        print(output.stdout)

Took 00:00:00.00 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:07.90 to generate walks
Took 00:00:00.02 to train embeddings

Took 00:00:00.00 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:07.83 to generate walks
Took 00:00:00.03 to train embeddings

Took 00:00:00.00 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:06.36 to generate walks
Took 00:00:00.02 to train embeddings

Took 00:00:00.00 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:07.80 to generate walks
Took 00:00:00.02 to train embeddings

Took 00:00:00.00 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:07.63 to generate walks
Took 00:00:00.04 to train embeddings

Took 00:00:00.00 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:06.33 to generate walks
Took 00:00:00.02 to train embeddings



In [ ]:
list_emb = os.listdir(output_path)

In [ ]:
for col in granger_list:

    dfx_granger = pd.pivot_table(df_granger, values=col, index=['var1'], columns=['var2']).fillna(0)
    dfx_granger.index.name = None
    dfx_granger.columns.name = None

    dfx_granger.index = dfx_granger.index.map(inv_map_var)
    dfx_granger.columns = [inv_map_var[col] for col in dfx_granger.columns]

    for dim in [2, 3]:
        filename = f"{col}_dim{dim}.emb"
        if filename in list_emb:
            df_temp = pd.read_csv(f"{output_path}/{filename}", delimiter=' ', skiprows = 1, header = None, index_col = [0])
            df_temp.columns = [f'N2VP_dim{i}of{dim}' for i in range(1, dim+1)]

            # df_temp = df_temp.reset_index().rename(columns = {'index':'var', 0:'var'})
            # df_temp['var'] = df_temp['var'].map(inv_map_var)
            df_temp.index = df_temp.index.map(inv_map_var)

            dfx_granger = dfx_granger.merge(df_temp, left_index=True, right_index=True, how='left')

    dfx_granger.index.name = f"{col}"
    dict_df[f"{col}"] = dfx_granger

In [ ]:
# with pd.ExcelWriter(f"Data/Embedding.xlsx") as writer:
#     for sheet_name, df in dict_df.items():
#         df.to_excel(writer, sheet_name=sheet_name)

In [ ]:
dict_df.keys()

dict_keys(['Pearson_Weighted', 'Pearson_Positive', 'Pearson_AbsGreater005', 'Pearson_AbsGreater01', 'Pearson_AbsGreater02', 'Pearson_AbsGreater05', 'Spearman_Weighted', 'Spearman_Positive', 'Spearman_AbsGreater005', 'Spearman_AbsGreater01', 'Spearman_AbsGreater02', 'Spearman_AbsGreater05', 'Kendall_Weighted', 'Kendall_Positive', 'Kendall_AbsGreater005', 'Kendall_AbsGreater01', 'Kendall_AbsGreater02', 'Kendall_AbsGreater05', 'Granger_6D', 'Granger_11D', 'Granger_VL'])

# [3] BiCop

In [ ]:
df_cop = pd.read_excel("/content/drive/MyDrive/graph_analytics/ThaiCapitalMarket/GraphEmbeding/Data/BivarCopula_Sig.xlsx")

In [ ]:
df_cop

,Y,X,BivarCopula,BivarCopula_L1,BivarCopula_2D
0,Thai-Equity-Set,Thai-Equity-MAI,False,True,True
1,Thai-Equity-MAI,Thai-Equity-Set,False,False,False
2,Thai-Equity-Set,Thai-Zero-Short,False,False,False
3,Thai-Zero-Short,Thai-Equity-Set,False,False,False
4,Thai-Equity-Set,Thai-Zero-Long,False,False,False
...,...,...,...,...,...
177,EM-Asia-FX,US-Bond,False,False,False
178,US-Bond,US-FX,False,True,True
179,US-FX,US-Bond,False,False,False
180,EM-Asia-FX,US-FX,False,False,False


In [ ]:
map_var2 = {'Thai-Equity-Set': 1,
            'Thai-Equity-MAI': 2,
            'Thai-Zero-Short': 3,
            'Thai-Zero-Long': 4,
            'Thai-Bond-Corp': 5,
            'Thai-FX': 6,
            'Thai-Equity-Flow': 7,
            'Thai-Bond-Flow-NR': 8,
            'EM-Asia-Equity': 9,
            'US-Equity': 10,
            'EM-Bond': 11,
            'US-Bond': 12,
            'EM-Asia-FX': 13,
            'US-FX': 14}

In [ ]:
# df_cop['var1'] = df_cop['Y'].map(map_var2)
# df_cop['var2'] = df_cop['X'].map(map_var2)

df_cop['var1'] = df_cop['X'].map(map_var2)
df_cop['var2'] = df_cop['Y'].map(map_var2)

In [ ]:
df_cop = df_cop.sort_values(['var1', 'var2'], ignore_index=True)

In [ ]:
df_cop

,Y,X,BivarCopula,BivarCopula_L1,BivarCopula_2D,var1,var2
0,Thai-Equity-MAI,Thai-Equity-Set,False,False,False,1,2
1,Thai-Zero-Short,Thai-Equity-Set,False,False,False,1,3
2,Thai-Zero-Long,Thai-Equity-Set,False,False,False,1,4
3,Thai-Bond-Corp,Thai-Equity-Set,False,False,False,1,5
4,Thai-FX,Thai-Equity-Set,False,False,False,1,6
...,...,...,...,...,...,...,...
177,EM-Asia-Equity,US-FX,True,False,True,14,9
178,US-Equity,US-FX,False,False,False,14,10
179,EM-Bond,US-FX,True,True,True,14,11
180,US-Bond,US-FX,False,True,True,14,12


In [ ]:
path_save = "Data/input"
os.makedirs(path_save, exist_ok=True)

In [ ]:
bicop_list = ['BivarCopula', 'BivarCopula_L1', 'BivarCopula_2D']

In [ ]:
for col in bicop_list:
    df_cop[df_cop[col]][['var1', 'var2']].to_csv(f"{path_save}/{col}.edg", sep = '\t', header = False, index = False)

In [ ]:
path_output = "Data/output"
os.makedirs(path_output, exist_ok=True)

In [ ]:
mode = "DenseOTF"
input_path = "/content/Data/input"
output_path = "/content/Data/output"

for dim in [2, 3]:
    for col in bicop_list:
        command_str = f"pecanpy --input {input_path}/{col}.edg --output {output_path}/{col}_dim{dim}.emb --mode {mode} --dimensions {dim} --directed"
        
        output = subprocess.run(command_str, shell=True, stdout=subprocess.PIPE, text=True)
        print(output.stdout)

Took 00:00:00.00 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:07.86 to generate walks
Took 00:00:00.02 to train embeddings

Took 00:00:00.00 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:07.71 to generate walks
Took 00:00:00.05 to train embeddings

Took 00:00:00.00 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:06.32 to generate walks
Took 00:00:00.02 to train embeddings

Took 00:00:00.00 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:08.00 to generate walks
Took 00:00:00.02 to train embeddings

Took 00:00:00.00 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:07.59 to generate walks
Took 00:00:00.04 to train embeddings

Took 00:00:00.00 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:06.33 to generate walks
Took 00:00:00.02 to train embeddings



In [ ]:
list_emb = os.listdir(output_path)

In [ ]:
# dict_df = {}

for col in bicop_list:

    dfx_cop = pd.pivot_table(df_cop, values=col, index=['var1'], columns=['var2']).fillna(0)
    dfx_cop.index.name = None
    dfx_cop.columns.name = None

    dfx_cop.index = dfx_cop.index.map(inv_map_var)
    dfx_cop.columns = [inv_map_var[col] for col in dfx_cop.columns]

    for dim in [2, 3]:
        filename = f"{col}_dim{dim}.emb"
        if filename in list_emb:
            df_temp = pd.read_csv(f"{output_path}/{filename}", delimiter=' ', skiprows = 1, header = None, index_col = [0])
            df_temp.columns = [f'N2VP_dim{i}of{dim}' for i in range(1, dim+1)]

            # df_temp = df_temp.reset_index().rename(columns = {'index':'var', 0:'var'})
            # df_temp['var'] = df_temp['var'].map(inv_map_var)
            df_temp.index = df_temp.index.map(inv_map_var)

            dfx_cop = dfx_cop.merge(df_temp, left_index=True, right_index=True, how='left')

    dfx_cop.index.name = f"{col}"
    dict_df[f"{col}"] = dfx_cop

In [ ]:
with pd.ExcelWriter(f"Data/Embedding.xlsx") as writer:
    for sheet_name, df in dict_df.items():
        df.to_excel(writer, sheet_name=sheet_name)

In [ ]:
dict_df.keys()

dict_keys(['Pearson_Weighted', 'Pearson_Positive', 'Pearson_AbsGreater005', 'Pearson_AbsGreater01', 'Pearson_AbsGreater02', 'Pearson_AbsGreater05', 'Spearman_Weighted', 'Spearman_Positive', 'Spearman_AbsGreater005', 'Spearman_AbsGreater01', 'Spearman_AbsGreater02', 'Spearman_AbsGreater05', 'Kendall_Weighted', 'Kendall_Positive', 'Kendall_AbsGreater005', 'Kendall_AbsGreater01', 'Kendall_AbsGreater02', 'Kendall_AbsGreater05', 'Granger_6D', 'Granger_11D', 'Granger_VL', 'BivarCopula', 'BivarCopula_L1', 'BivarCopula_2D'])

# Plot 2D

In [ ]:
list2d = [e for e in os.listdir("Data/output/") if "dim2" in e]

In [ ]:
list2d

['Pearson_AbsGreater005_dim2.emb',
 'Spearman_AbsGreater005_dim2.emb',
 'Kendall_Weighted_dim2.emb',
 'Pearson_Positive_dim2.emb',
 'Kendall_AbsGreater01_dim2.emb',
 'Kendall_AbsGreater02_dim2.emb',
 'BivarCopula_dim2.emb',
 'Pearson_Weighted_dim2.emb',
 'Pearson_AbsGreater02_dim2.emb',
 'Kendall_Positive_dim2.emb',
 'Granger_6D_dim2.emb',
 'Granger_VL_dim2.emb',
 'Spearman_Positive_dim2.emb',
 'Pearson_AbsGreater01_dim2.emb',
 'Spearman_AbsGreater05_dim2.emb',
 'BivarCopula_L1_dim2.emb',
 'Pearson_AbsGreater05_dim2.emb',
 'BivarCopula_2D_dim2.emb',
 'Spearman_Weighted_dim2.emb',
 'Granger_11D_dim2.emb',
 'Spearman_AbsGreater01_dim2.emb',
 'Spearman_AbsGreater02_dim2.emb',
 'Kendall_AbsGreater005_dim2.emb']

In [ ]:
path_fig = "Data/fig"
os.makedirs(path_fig, exist_ok = True)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# df2 = pd.read_csv(f"/content/Data/output/kendall_abs_gt025_dim2.emb", delimiter=' ', skiprows = 1, header = None, index_col = [0], names=['x', 'y'])
# df2 = df2.reset_index().rename(columns = {'index':'var'})
# df2['var'] = df2['var'].map(inv_map_var)

In [ ]:
for e in list2d:
    filename = e.split(".")[0]
    df2 = pd.read_csv(f"/content/Data/output/{e}", delimiter=' ', skiprows = 1, header = None, index_col = [0], names=['x', 'y'])
    df2 = df2.reset_index().rename(columns = {'index':'var'})
    df2['var'] = df2['var'].map(inv_map_var)

    fig, ax = plt.subplots(figsize = (14, 8))
    ax.scatter(df2['x'], df2['y'])

    for i, row in df2.iterrows():
        ax.annotate(row['var'], (row['x'], row['y']), fontsize=14)

    plt.title(filename, fontsize=18)
    fig.savefig(f"{path_fig}/{filename}.png")
    plt.close(fig)

In [ ]:
# for e in os.listdir("/content/Data/output"):
#     df2 = pd.read_csv(f"/content/Data/output/{e}", delimiter=' ', skiprows = 1, header = None, index_col = [0])
#     if df2.shape[1] == 2:
#         df2.columns = ['x', 'y']
#     else:
#         df2.columns = ['x', 'y', 'z']
#     df2 = df2.reset_index().rename(columns = {'index':'var', 0:'var'})
#     df2['var'] = df2['var'].map(inv_map_var)

#     name = e.split('.')[0]
#     df2.to_csv(f"/content/Data/output/{name}.csv", index=False)

In [ ]:
# dfx3['var1_name'] = dfx3['var1'].map(inv_map_var)
# dfx3['var2_name'] = dfx3['var2'].map(inv_map_var)

In [ ]:
# dfx3[['var1_name', 'var2_name', 'var1', 'var2', 'pearson', 'spearman', 'kendall']].to_csv("Data/Correlations.csv", index=False)

In [ ]:
!zip -r N2VP_230110.zip Data/*

  adding: Data/Embedding.xlsx (deflated 9%)
  adding: Data/fig/ (stored 0%)
  adding: Data/fig/BivarCopula_2D_dim2.png (deflated 20%)
  adding: Data/fig/Kendall_AbsGreater01_dim2.png (deflated 20%)
  adding: Data/fig/Spearman_Positive_dim2.png (deflated 21%)
  adding: Data/fig/Pearson_AbsGreater02_dim2.png (deflated 22%)
  adding: Data/fig/Spearman_AbsGreater005_dim2.png (deflated 20%)
  adding: Data/fig/Pearson_Positive_dim2.png (deflated 21%)
  adding: Data/fig/Granger_6D_dim2.png (deflated 21%)
  adding: Data/fig/BivarCopula_dim2.png (deflated 22%)
  adding: Data/fig/BivarCopula_L1_dim2.png (deflated 21%)
  adding: Data/fig/Kendall_AbsGreater005_dim2.png (deflated 21%)
  adding: Data/fig/Granger_VL_dim2.png (deflated 20%)
  adding: Data/fig/Pearson_AbsGreater005_dim2.png (deflated 19%)
  adding: Data/fig/Pearson_Weighted_dim2.png (deflated 21%)
  adding: Data/fig/Pearson_AbsGreater01_dim2.png (deflated 20%)
  adding: Data/fig/Kendall_AbsGreater02_dim2.png (deflated 22%)
  adding: Da

# [-] Trial

In [ ]:
!pecanpy --input /content/Data/input/pearson_w.edg --output /content/Data/output/pearson_w_dim3.emb --mode PreComp --dimensions 3 --weighted --extend
!pecanpy --input /content/Data/input/pearson_gt0.edg --output /content/Data/output/pearson_gt0_dim3.emb --mode PreComp --dimensions 3
!pecanpy --input /content/Data/input/pearson_abs_gt05.edg --output /content/Data/output/pearson_abs_gt05_dim3.emb --mode PreComp --dimensions 3
!pecanpy --input /content/Data/input/pearson_abs_gt025.edg --output /content/Data/output/pearson_abs_gt025_dim3.emb --mode PreComp --dimensions 3

!pecanpy --input /content/Data/input/spearman_w.edg --output /content/Data/output/spearman_w_dim3.emb --mode PreComp --dimensions 3 --weighted --extend
!pecanpy --input /content/Data/input/spearman_gt0.edg --output /content/Data/output/spearman_gt0_dim3.emb --mode PreComp --dimensions 3
!pecanpy --input /content/Data/input/spearman_abs_gt05.edg --output /content/Data/output/spearman_abs_gt05_dim3.emb --mode PreComp --dimensions 3
!pecanpy --input /content/Data/input/spearman_abs_gt025.edg --output /content/Data/output/spearman_abs_gt025_dim3.emb --mode PreComp --dimensions 3

!pecanpy --input /content/Data/input/kendall_w.edg --output /content/Data/output/kendall_w_dim3.emb --mode PreComp --dimensions 3 --weighted --extend
!pecanpy --input /content/Data/input/kendall_gt0.edg --output /content/Data/output/kendall_gt0_dim3.emb --mode PreComp --dimensions 3
!pecanpy --input /content/Data/input/kendall_abs_gt05.edg --output /content/Data/output/kendall_abs_gt05_dim3.emb --mode PreComp --dimensions 3
!pecanpy --input /content/Data/input/kendall_abs_gt025.edg --output /content/Data/output/kendall_abs_gt025_dim3.emb --mode PreComp --dimensions 3

Took 00:00:00.00 to load Graph
Took 00:00:07.44 to pre-compute transition probabilities
Took 00:00:04.68 to generate walks
Took 00:00:00.01 to train embeddings
Took 00:00:00.00 to load Graph
Took 00:00:05.78 to pre-compute transition probabilities
Took 00:00:04.56 to generate walks
Took 00:00:00.01 to train embeddings
Took 00:00:00.00 to load Graph
Took 00:00:05.70 to pre-compute transition probabilities
Took 00:00:04.50 to generate walks
Took 00:00:00.01 to train embeddings
Took 00:00:00.00 to load Graph
Took 00:00:05.69 to pre-compute transition probabilities
Took 00:00:04.60 to generate walks
Took 00:00:00.01 to train embeddings
Took 00:00:00.00 to load Graph
Took 00:00:07.38 to pre-compute transition probabilities
Took 00:00:04.70 to generate walks
Took 00:00:00.01 to train embeddings
Took 00:00:00.00 to load Graph
Took 00:00:05.69 to pre-compute transition probabilities
Took 00:00:04.51 to generate walks
Took 00:00:00.01 to train embeddings
Took 00:00:00.00 to load Graph
Took 00:0

In [ ]:
!pecanpy --input /content/Data/input/pearson_w.edg --output /content/Data/output/pearson_w_dim2.emb --mode PreComp --dimensions 2 --weighted --extend
!pecanpy --input /content/Data/input/pearson_gt0.edg --output /content/Data/output/pearson_gt0_dim2.emb --mode PreComp --dimensions 2
!pecanpy --input /content/Data/input/pearson_abs_gt05.edg --output /content/Data/output/pearson_abs_gt05_dim2.emb --mode PreComp --dimensions 2
!pecanpy --input /content/Data/input/pearson_abs_gt025.edg --output /content/Data/output/pearson_abs_gt025_dim2.emb --mode PreComp --dimensions 2

!pecanpy --input /content/Data/input/spearman_w.edg --output /content/Data/output/spearman_w_dim2.emb --mode PreComp --dimensions 2 --weighted --extend
!pecanpy --input /content/Data/input/spearman_gt0.edg --output /content/Data/output/spearman_gt0_dim2.emb --mode PreComp --dimensions 2
!pecanpy --input /content/Data/input/spearman_abs_gt05.edg --output /content/Data/output/spearman_abs_gt05_dim2.emb --mode PreComp --dimensions 2
!pecanpy --input /content/Data/input/spearman_abs_gt025.edg --output /content/Data/output/spearman_abs_gt025_dim2.emb --mode PreComp --dimensions 2

!pecanpy --input /content/Data/input/kendall_w.edg --output /content/Data/output/kendall_w_dim2.emb --mode PreComp --dimensions 2 --weighted --extend
!pecanpy --input /content/Data/input/kendall_gt0.edg --output /content/Data/output/kendall_gt0_dim2.emb --mode PreComp --dimensions 2
!pecanpy --input /content/Data/input/kendall_abs_gt05.edg --output /content/Data/output/kendall_abs_gt05_dim2.emb --mode PreComp --dimensions 2
!pecanpy --input /content/Data/input/kendall_abs_gt025.edg --output /content/Data/output/kendall_abs_gt025_dim2.emb --mode PreComp --dimensions 2

Took 00:00:00.00 to load Graph
Took 00:00:08.81 to pre-compute transition probabilities
Took 00:00:05.61 to generate walks
Took 00:00:00.01 to train embeddings
Took 00:00:00.00 to load Graph
Took 00:00:08.88 to pre-compute transition probabilities
Took 00:00:05.98 to generate walks
Took 00:00:00.02 to train embeddings
Took 00:00:00.00 to load Graph
Took 00:00:07.05 to pre-compute transition probabilities
Took 00:00:04.55 to generate walks
Took 00:00:00.01 to train embeddings
Took 00:00:00.00 to load Graph
Took 00:00:05.82 to pre-compute transition probabilities
Took 00:00:04.57 to generate walks
Took 00:00:00.01 to train embeddings
Took 00:00:00.00 to load Graph
Took 00:00:07.46 to pre-compute transition probabilities
Took 00:00:04.70 to generate walks
Took 00:00:00.01 to train embeddings
Took 00:00:00.00 to load Graph
Took 00:00:05.70 to pre-compute transition probabilities
Took 00:00:04.55 to generate walks
Took 00:00:00.01 to train embeddings
Took 00:00:00.00 to load Graph
Took 00:0